In [42]:
# Import libraries
import os
import requests
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow_metadata.proto.v0 import schema_pb2
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)


TF version: 2.13.0
TFDV version: 1.14.0


# 2. Descripción del Conjunto de Datos

In [3]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [4]:
data = pd.read_csv(_data_filepath)

In [5]:
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

## Inicializar el repositorio de DVC y añadir los datos

In [8]:
#!dvc init

In [8]:
!git commit -m 'Initialize DVC'

[main 06c1f44] Initialize DVC
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore


In [9]:
!dvc add data/

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/work/.dvc/cache/files/md5'| |0/? [00:00<?,    ?files/s]
                                                                                
!
  0%|          |Adding data to cache                  0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /work/data               0/2 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 24.26file/s]

To track the changes with git, run:

	git add .gitignore data.dvc

To enable auto staging, run:

	dvc config core.autostage true


# 3. Seleción de características

In [9]:
# Selecciona solo las características numéricas
data_numeric_features = data.select_dtypes(include=['number'])
# Separar target de features
X = data_numeric_features.drop('Cover_Type', axis=1)  # Assuming 'Cover_Type' is the target
y = data['Cover_Type']

# Feature selection
selector = SelectKBest(f_classif, k=8) 
X_new = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
features_retain = {feature: (feature in selected_features) for feature in X.columns}

In [10]:
features_retain

{'Elevation': True,
 'Aspect': False,
 'Slope': True,
 'Horizontal_Distance_To_Hydrology': True,
 'Vertical_Distance_To_Hydrology': True,
 'Horizontal_Distance_To_Roadways': True,
 'Hillshade_9am': True,
 'Hillshade_Noon': True,
 'Hillshade_3pm': False,
 'Horizontal_Distance_To_Fire_Points': True}

In [25]:
data_selected_features = data[list(selected_features) + 
                              ['Wilderness_Area', 'Soil_Type', 'Cover_Type']]

In [26]:
data_selected_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Slope                               116203 non-null  int64 
 2   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 3   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 4   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 5   Hillshade_9am                       116203 non-null  int64 
 6   Hillshade_Noon                      116203 non-null  int64 
 7   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 8   Wilderness_Area                     116203 non-null  object
 9   Soil_Type                           116203 non-null  object
 10  Cover_Type                          116203 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 9.

In [27]:
data_selected_features.to_csv(_data_filepath,index=False)

# 4. Data Pipeline

## 4.1 Configurar el contexto interactivo

In [13]:
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

In [15]:
_pipeline_root = './pipeline/'
context = InteractiveContext(pipeline_root=_pipeline_root)

## 4.2 Generando Ejemplos Con CsvExampleGen

In [28]:
example_gen = CsvExampleGen(input_base = _data_root)

In [29]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 4
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 4.3 Estadísticas

In [30]:
# Crear una instancia de StatisticsGen con ExampleGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

#Ejecutar el componente
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

## 4.4 Inferir el esquema 

In [32]:
schema_gen = SchemaGen(statistics = statistics_gen.outputs['statistics'],)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 6
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [33]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## 4.5 Curando el esquema

In [34]:
schema_path = schema_gen.outputs['schema']._artifacts[0].uri + '/schema.pbtxt'

In [40]:
schema = tfdv.load_schema_text(schema_path)

In [43]:
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(name='Hillshade_9am', min=0, max=255))
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(name='Hillshade_Noon', min=0, max=255))
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(name='Slope', min=0, max=90))